In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
%pylab inline


Populating the interactive namespace from numpy and matplotlib


###Definicion de variables


In [2]:
rll=0.139 # Radio geométrico de las llantas en traseras (m)
refc=0.98*rll # Radio efectivo llantas traseras (m)
rin=0.0631 #radio interno del neumático (m)

# Datos eje llantas
reje=0.02 # Radio eje de las llantas (m)
meje=3.832 # Masa del eje (kg)


mllt=1.43 # Masa llantas traseras (kg)
mlld=1.20 # Masa llantas delanteras (kg)
rlld=0.129 # Radio llantas delanteras (m)
rind=0.0631 # Radio rin llantas delanteras (m)
mbt=0.7*mllt # El porcentaje de la masa total que pesa la parte "banda" de la llanta trasera
mdt=0.3*mllt # El porcentaje de la masa total que pesa la parte "disco" de la llanta trasera
mbd=0.7*mlld # El porcentaje de la masa total que pesa la parte "banda" de la llanta delantera
mdd=0.3*mlld # El porcentaje de la masa total que pesa la parte "disco" de la llanta delantera
illd=(mbd*rlld)+(mdd*(rlld**2-rind**2))/2 # Inercia 1 llanta delantera (kg-m^2)
illt=(mbt*rll)+(mdt*(rll**2-rin**2))/2 # Inercia 1 llanta trasera (kg-m^2)

ieje=(meje*reje**2)/2.0 # Inercia del eje de las llantas (kg-m^2)

theta=0##angulo de inclinacion de la pista
nt=0.95 #eficiencia de la transmision
nc=2.25 #relacion de transmision
r=refc#radio de las llantas
m=147 #masa vehiculo
rho=0.95 #densidad del aire
af=0.575 #area frontal del vehiculo
cd=0.804 #coeficiente de arrastre
inw=illt#inercia de las llantas
ind=ieje#inercia del eje
ine=5.2E-3#inercia del motor
pi=3.1459265 #coeficiente de arrastre
mr=(inw+ind+(ine*(nc**2)))/(r**2)##calculo de la masa rotacional
mtot=m+mr##calculo de la masa total

###fuerza de rodadura

In [3]:
def fr(vel):#funcion para calcular la fuerza de rodadura a una velocidad dada
    vkmh=vel*3.6
    w=m*9.8
    fr=0.01*(1+(vkmh/100.0)) 
    return w*fr*cos(theta)

###Fuerza de frenado

In [4]:
def ffren(vel):#funcion para calcular el torque de frenado a una velocidad dada 
 frenadomax=-20
 res=frenadomax/r
 return(res)

###fuerza de arrastre



In [5]:
def da(vel):#funcion para calcular la fuerza de arrastre a una velocidad dada
 resp=0.5*rho*af*cd*(vel**2)
 return (resp)


###torque motor

In [6]:
def torq(vel):#funcion para calcular el torque generado a una velocidad dada
 w=vel*nc/r
 wrpm=w*60/(2*pi)
 c2=(-4.368628107/(10**8))
 c3=(8.598594866/(10**5))
 c4=1.57456788
 c5=4.8948056
 pot=((c2*(wrpm**3))+(c3*(wrpm**2))+(c4*wrpm)+c5)
 resp=pot/w
 return (resp)

In [7]:
def derivada(velocidad):#funcion para determinar la aceleracion a una velocidad dada durante la aceleracion
  rpm=(velocidad/(nc*r))*60/(2*pi)  
  if rpm>5000.0:
   resp=0# se restrigne el aumento de velocidad hasta la velocidad maxima del motor 
  else:
   trac=torq(velocidad)*nc*nt/r
   resp=(trac-da(velocidad)-fr(velocidad))/mtot 
  return resp


In [8]:
def derivadafren(velocidad):#funcion para determinar la aceleracion a una velocidad dada durante el frenado
 trac=torq(velocidad)*nc*nt/r
 if velocidad>0.01:#se restringe la aceleracion para que llegado a cero no tenga velocidad negativa
  resp=(ffren(velocidad)-da(velocidad)-fr(velocidad))/mtot 
 else:
  resp=0      
 return resp

In [9]:
 def pendiente(x, t):#se genera el sistema de dos ecuaciones (primera y segunda derivada) para utilizar con odeint en la aceeracion
  pos, vel = x
  dxdt = [vel,derivada(vel)]
  return dxdt

In [10]:
 def pendientefren(x, t):#se genera el sistema de dos ecuaciones (primera y segunda derivada) para utilizar con odeint en el frenado
  pos, vel = x
  dxdt = [vel,derivadafren(vel)]
  return dxdt

In [11]:
inicial=[0.0,0.1]#funcion para variar la relacion de transmision y cacular la velocidad maxima y tiempo hasta velmax
variableini=nc
variable= np.linspace(0.4,4, 101)
tiemp = np.linspace(0,400, 4001)
long = np.linspace(0,4001, 4002)
long=long.astype(int)
velmax=[]
tmax=[]
indice=0
for i in variable:
 boolean=1.0
 nc=i
 resp = odeint(pendiente,inicial, tiemp) 
 velo=resp[:,1]
 velmax.append(np.amax(velo))
 for j in long:
    if boolean==1.0:
      val1=round(velo[j], 3)
      val2=round(velmax[indice], 3)
      if val1==val2:
       tmax.append(tiemp[j])
       boolean=0.0
 indice=indice+1

nc=variableini
plt.plot(variable,velmax)

pylab.legend(loc='center right')#grafica de velocidad maxima vs relacion de transmision para determinar la mejor relacion  
plt.title('Grafica 1. Velocidad maxima vs relacion de transmicion')
pylab.ylabel("velocidad maxima alcanzada [m/s]")
pylab.xlabel("Relacion de transmision")
plt.grid()
pylab.savefig(os.path.join('graf1.png'), dpi=600)
plt.close()

In [12]:
inicial=[0.0,0.1]#funcion para generar las graficas de posicion y velocidad vs tiempo para arranque y frenado
cont=0
longit=[]
posicion=[]
velocidad=[]
tiempo=[]
tacel= np.asarray([0.0,0.1])
longit.append(cont)
posicion.append(inicial[0])
velocidad.append(inicial[1])
tiempo.append(0.0)
while (cont ==0 ):
 solve = odeint(pendiente,inicial,tacel) 
 vel=solve[:,1]
 pos=solve[:,0]
 if round(vel[0], 3)==round(vel[1], 3):
   cont=1
 else:
  posicion.append(pos[1]) 
  velocidad.append(vel[1])
  tiempo.append(tacel[1])
  tacel=tacel+0.1
  inicial=[posicion[-1],velocidad[-1]]

while (cont ==1 ):
 solve = odeint(pendientefren,inicial,tacel) 
 vel=solve[:,1]
 pos=solve[:,0]
 if round(vel[0], 3)==round(vel[1], 3):
   cont=2
 else:
  posicion.append(pos[1]) 
  velocidad.append(vel[1])
  tiempo.append(tacel[1])
  tacel=tacel+0.1
  inicial=[posicion[-1],velocidad[-1]]

  
plt.plot(tiempo, velocidad) 
plt.title('Grafica 2. Velocidad vs tiempo para arranque y frenado nc=2.25')
pylab.ylabel("Velocidad [m/s]")
pylab.xlabel("Tiempo  [s]")
plt.grid()
pylab.savefig(os.path.join('graf2.png'), dpi=600)
plt.close()

plt.plot(tiempo, posicion) 
plt.title('Grafica 3. Posicion vs tiempo para arranque y frenado nc=2.25')
pylab.ylabel("Posicion [m]")
pylab.xlabel("Tiempo  [s]")
plt.grid()
pylab.savefig(os.path.join('graf3.png'), dpi=600)
plt.close()


In [13]:
inicial2=[0.0,0.1]#graficas de velocidad y posicion vs tiempo para distintas relaciones de transmision
variableini=nc
relaciones= [0.5,1,1.5,2,2.25,3.5]
tiemp2 = np.linspace(0,120, 1201)
velocidad=[]
posicion=[]
for j in relaciones:
 nc=j
 solucion = odeint(pendiente,inicial2,tiemp2) 
 posicion.append(solucion[:,0])
 velocidad.append(solucion[:,1])
nc=variableini
plt.plot(tiemp2, velocidad[0],label="0.5")
plt.plot(tiemp2, velocidad[1],label="1.0")
plt.plot(tiemp2, velocidad[2],label="1.5")
plt.plot(tiemp2, velocidad[3],label="2.0")
plt.plot(tiemp2, velocidad[4],label="2.25")
plt.plot(tiemp2, velocidad[5],label="3.5")
pylab.legend(loc='lower right')  
plt.title('Grafica 4. velocidad vs tiempo para distintas relaciones de transmision')
pylab.ylabel("Velocidad [m/s]")
pylab.xlabel("Tiempo  [s]")
plt.grid()
pylab.savefig(os.path.join('graf4.png'), dpi=600)
plt.close()

plt.plot(tiemp2, posicion[0],label="0.5")
plt.plot(tiemp2, posicion[1],label="1.0")
plt.plot(tiemp2, posicion[2],label="1.5")
plt.plot(tiemp2, posicion[3],label="2.0")
plt.plot(tiemp2, posicion[4],label="2.25")
plt.plot(tiemp2, posicion[5],label="3.5")
pylab.legend(loc='lower right')  
plt.title('Grafica 5. Posicion vs tiempo para distintas relaciones de transmision')
pylab.ylabel("posicion lineal [m]")
pylab.xlabel("Tiempo  [s]")
plt.grid()
pylab.savefig(os.path.join('graf5.png'), dpi=600)
plt.close()


In [14]:
inicial2=[0.0,0.1]#graficas de velocidad y posicion vs tiempo para distintas masas del vehiculo
masas= [100,150,175,200,225]
tiemp2 = np.linspace(0,150, 1501)
velocidad=[]
posicion=[]
for j in masas:
 m=j
 mtot=m+mr
 solucion = odeint(pendiente,inicial2,tiemp2) 
 posicion.append(solucion[:,0])
 velocidad.append(solucion[:,1])
m=variableini
mtot=m+mr
plt.plot(tiemp2, velocidad[0],label="100 kg")
plt.plot(tiemp2, velocidad[1],label="150 kg")
plt.plot(tiemp2, velocidad[2],label="175 kg")
plt.plot(tiemp2, velocidad[3],label="200 kg")
plt.plot(tiemp2, velocidad[4],label="225 kg")
pylab.legend(loc='lower right')  
plt.title('Grafica 6. velocidad vs tiempo para distintas masas del kart')
pylab.ylabel("Velocidad [m/s]")
pylab.xlabel("Tiempo  [s]")
plt.grid()
pylab.savefig(os.path.join('graf6.png'), dpi=600)
plt.close()

plt.plot(tiemp2, posicion[0],label="100 kg")
plt.plot(tiemp2, posicion[1],label="150 kg")
plt.plot(tiemp2, posicion[2],label="175 kg")
plt.plot(tiemp2, posicion[3],label="200 kg")
plt.plot(tiemp2, posicion[4],label="225 kg")
pylab.legend(loc='lower right')  
plt.title('Grafica 7. posicion vs tiempo para distintas masas del kart')
pylab.ylabel("posicion lineal [m]")
pylab.xlabel("Tiempo  [s]")
plt.grid()
pylab.savefig(os.path.join('graf7.png'), dpi=600)
plt.close()


# Análisis del kart

##relación de transmisión optima

A continuación, se presenta la gráfica generada de velocidad máxima vs relación de transmisión con el propósito de observar el comportamiento de estas dos variables una respecto a la otra

<img src="graf1/RMSN.png">

se puede apreciar que la velocidad máxima aumenta al aumentar la relación de transmisión hasta llegar a un punto de inflexión a partir del cual decae. este comportamiento es causado por el hecho de que a relaciones muy bajas el torque transmitido es muy bajo para lograr vencer las fuerzas de rodadura y arrastre a velocidades elevadas mientras que con relaciones de transmisión demasiado altas se genera un torque elevado pero la velocidad máxima se restringe por la velocidad máxima del motor.
De la gráfica 1 se puede apreciar que la relación que genera una velocidad máxima es de 2.25 y seria la relación a utilizar si se desea que el kart pueda alcanzar velocidades más altas. 

##Graficas de aceleración y desaceleración

Tomando como punto de partida que la velocidad máxima del vehículo se obtiene con una relación de transmisión de 2.25 se utilizó este valor para simular la aceleración y posterior frenado del vehículo. las gráficas obtenidas se presentan a continuación 

<img src="graf2.png">
<img src="graf3.png">

de las gráficas es posible apreciar que el vehículo tarda alrededor de 55 segundos en alcanzar su velocidad máxima y recorre alrededor de 1.35km antes de volver a detenerse.

##Graficas de velocidad y posición vs relación de transmisión

Con el propósito de tener una visión más amplia del comportamiento del kart respecto a la relación de transmisión y no únicamente conocer la velocidad máxima. se generaron graficas de velocidad y distancia contra tiempo para distintas relaciones de transmisión las cuales se presentan a continuación.

<img src="graf4.png">
<img src="graf5.png">

de la gráfica 4 se puede apreciar que entre más baja es la relación de velocidad no solo se tiene una menor velocidad máxima sino también toma un tiempo mayor alcanzar dicha velocidad, esto se debe a que para este tipo de relaciones el torque que se presenta en las llantas es muy bajo y por la segunda ley de newton se presentan aceleraciones bajas.

también se puede apreciar que entre más elevada sea la relación de transmisión la aceleración es más alta y por ende alcanza esta velocidad más rápidamente, sin embargo, después de la relación de 2.25 observada anteriormente esta velocidad máxima se reduce.

respecto a la gráfica 5 se puede apreciar que entre mayor sea la relación de transmisión más alta será la distancia recorrida en los primeros segundos de movimiento, por ejemplo, la distancia recorrida por e kart para una relación de 3.5 es mayor que la distancia para una relación de 2.25 durante los primeros cincuenta segundos.

lo anterior indica que dependiendo del tipo de carrera una relación más alta podría ser mejor. por ejemplo, si el circuito de carreras presenta rectas cortas donde difícilmente se pudiera alcanzar la velocidad máxima una relación de transmisión alta que genere aceleraciones mayores permitirá recorrer estas distancias cortas más rápidamente.

##Graficas de velocidad y posición vs peso

para analizar el efecto del peso sobre el comportamiento de kart se realizaron graficas de velocidad y distancia vs tiempo para distintos pesos del kart, los resultados se presentan a continuación.


<img src="graf6.png">
<img src="graf7.png">

las gráficas 6 y 7 presentan un comportamiento esperado de velocidad y distancia recorrida puesto que por la segunda ley de newton a menor masa y una relación de transmisión constante (mismo torque) el vehículo de menor masa se acelerará más y llegara a una velocidad máxima mayor, lo cual también se traduce en una longitud recorrida mayor en un periodo de tiempo dado. debido a esto sería valioso para aumentar el desempeño general del kart reducir el peso hasta el mínimo posible.
